## FinTech Vendor Scorecard for Micro-Lending

In [ ]:

import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch
from datetime import datetime
from google.colab import files
import os
import re
import shutil

# Define paths
data_path = "./data/preprocessed_data.csv"
model_path = "./amharic-ner-model"
output_report = "./vendor_scorecard.md"

# Create data directory if it doesn't exist
os.makedirs("./data", exist_ok=True)

# Load NER model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
id2label = model.config.id2label
label2id = model.config.label2id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Check for data file and handle upload
if not os.path.exists(data_path):
    print("Please upload preprocessed_data.csv")
    uploaded = files.upload()
    uploaded_file = list(uploaded.keys())[0] if uploaded else None
    if uploaded_file and os.path.exists(uploaded_file):
        # Move uploaded file to ./data/
        shutil.move(uploaded_file, data_path)
    else:
        raise FileNotFoundError("preprocessed_data.csv not found or upload failed.")

# Load preprocessed data
df = pd.read_csv(data_path)
# Rename 'text' to 'message' for consistency
df = df.rename(columns={'text': 'message'})

# Fallback regex for price extraction
def fallback_price_extraction(text):
    if not isinstance(text, str):
        return []
    matches = re.findall(r'(\d+\.?\d*)\s*ብር', text)
    return [float(m) for m in matches]

# Function to extract entities from text
def extract_entities(text):
    if not isinstance(text, str):
        return [], []
    predictions = ner_pipeline(text)
    products, prices = [], []
    current_product = []
    current_price = []
    for pred in predictions:
        entity = pred["entity"]
        word = pred["word"]
        if entity.startswith("B-Product"):
            if current_product:
                products.append(" ".join(current_product))
                current_product = []
            current_product.append(word)
        elif entity.startswith("I-Product"):
            current_product.append(word)
        elif entity.startswith("B-PRICE"):
            if current_price:
                prices.append(" ".join(current_price))
                current_price = []
            current_price.append(word)
        elif entity.startswith("I-PRICE"):
            current_price.append(word)
    if current_product:
        products.append(" ".join(current_product))
    if current_price:
        prices.append(" ".join(current_price))
    # Use regex fallback if NER fails to extract prices
    if not prices:
        prices = [str(p) + " ብር" for p in fallback_price_extraction(text)]
    return products, prices

# Function to parse price to float (in Birr)
def parse_price(price_str):
    try:
        match = re.search(r'(\d+\.?\d*)\s*ብር', price_str)
        if match:
            return float(match.group(1))
        return None
    except:
        return None

# Process vendor data
def analyze_vendor(channel, vendor_df):
    # Convert timestamps to datetime
    vendor_df['timestamp'] = pd.to_datetime(vendor_df['timestamp'], errors='coerce')
    vendor_df = vendor_df.dropna(subset=['timestamp'])

    # Posting Frequency (posts per week)
    min_date = vendor_df['timestamp'].min()
    max_date = vendor_df['timestamp'].max()
    days_diff = (max_date - min_date).days + 1
    weeks = days_diff / 7 if days_diff > 0 else 1
    posting_freq = len(vendor_df) / weeks

    # Average Views per Post (not available)
    avg_views = 0  # Placeholder due to missing 'views' column

    # Top Performing Post (not available, select most recent post)
    if not vendor_df.empty:
        top_post = vendor_df.sort_values('timestamp', ascending=False).iloc[0]
        top_views = 0  # Placeholder
        top_message = top_post['message']
        top_products, top_prices = extract_entities(top_message)
        top_product = top_products[0] if top_products else "Unknown"
        top_price = parse_price(top_prices[0]) if top_prices else None
    else:
        top_views, top_product, top_price, top_message = 0, "None", None, "None"

    # Average Price Point
    all_prices = []
    for message in vendor_df['message']:
        _, prices = extract_entities(message)
        for price in prices:
            parsed = parse_price(price)
            if parsed:
                all_prices.append(parsed)
    avg_price = np.mean(all_prices) if all_prices else 0

    # Lending Score (adjusted weights: 60% freq, 40% price due to missing views)
    max_freq = 10      # Assumed max posts/week
    max_price = 5000   # Assumed max price in Birr
    norm_freq = min(posting_freq / max_freq, 1) if max_freq > 0 else 0
    norm_price = min(avg_price / max_price, 1) if max_price > 0 else 0
    lending_score = (0.6 * norm_freq + 0.4 * norm_price) * 100

    return {
        "channel": channel,
        "posting_frequency": round(posting_freq, 2),
        "avg_views": avg_views,
        "top_post": {
            "views": top_views,
            "product": top_product,
            "price": top_price,
            "message": top_message
        },
        "avg_price_point": round(avg_price, 2),
        "lending_score": round(lending_score, 2)
    }

# Analyze all vendors
vendors = df['channel'].unique()
results = []
for channel in vendors:
    vendor_df = df[df['channel'] == channel]
    if not vendor_df.empty:
        result = analyze_vendor(channel, vendor_df)
        results.append(result)

# Generate Scorecard Report
report = "# Vendor Scorecard for Micro-Lending\n\n"
report += "## Overview\n"
report += "- **Objective**: Identify promising vendors for micro-lending based on activity and business profile.\n"
report += "- **Data Source**: Preprocessed Telegram posts (`preprocessed_data.csv`).\n"
report += "- **NER Model**: Fine-tuned `xlm-roberta-base` for entity extraction.\n\n"

report += "## Vendor Metrics\n"
for result in results:
    report += f"### Vendor: {result['channel']}\n"
    report += f"- **Posting Frequency**: {result['posting_frequency']} posts/week\n"
    report += f"- **Average Views per Post**: {result['avg_views']} views (not available in data)\n"
    report += f"- **Top Performing Post**:\n"
    report += f"  - Views: {result['top_post']['views']} (not available, showing most recent post)\n"
    report += f"  - Product: {result['top_post']['product']}\n"
    report += f"  - Price: {result['top_post']['price'] or 'Unknown'} Birr\n"
    report += f"  - Message: {result['top_post']['message'][:100] if isinstance(result['top_post']['message'], str) else 'None'}...\n"
    report += f"- **Average Price Point**: {result['avg_price_point']} Birr\n"
    report += f"- **Lending Score**: {result['lending_score']} (out of 100, adjusted for missing views)\n\n"

report += "## Notes\n"
report += "- **Data Limitations**: Missing 'views' column prevents calculation of view-based metrics. Top post is the most recent post.\n"
report += "- **NER Limitations**: The NER model has low performance (0% F1 for Product, 3.01% F1 for LOC), affecting price and product extraction accuracy. Regex fallback used for prices.\n"
report += "- **Normalization**: Lending Score uses assumed max values (frequency: 10 posts/week, price: 5,000 Birr) with adjusted weights (60% frequency, 40% price).\n"
report += "- **Recommendations**: Re-scrape data to include view counts, improve NER model with more labeled data, validate timestamps, and collect engagement metrics (e.g., replies).\n"

# Save report
with open(output_report, "w", encoding="utf-8") as f:
    f.write(report)

# Download report
files.download(output_report)

print("Vendor analytics completed. Scorecard saved and downloaded as 'vendor_scorecard.md'.")